<a href="https://colab.research.google.com/github/fleithpi/PROGRAMA-O-ORIENTADA-A-OBJETOS/blob/main/biblioteca_com_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# O código das classes Livro, Usuario e Biblioteca é o mesmo fornecido no prompt.
# Para manter o script auto-suficiente, vamos redefini-las.

# --- 1. CLASSE LIVRO (Com @property) ---
class Livro:
    def __init__(self, titulo: str, autor: str):
        # A remoção do espaço em branco no autor para o livro_1 é para padronizar
        self.__titulo = titulo.strip()
        self.__autor = autor.strip()
        self.__disponivel = True

    @property
    def titulo(self) -> str:
        return self.__titulo

    @property
    def autor(self) -> str:
        return self.__autor

    @property
    def disponivel(self) -> bool:
        return self.__disponivel

    def emprestar(self) -> bool:
        if self.__disponivel:
            self.__disponivel = False
            return True
        else:
            # A mensagem de erro é tratada na Biblioteca ou na interface para ser mais útil.
            return False

    def devolver(self) -> bool:
        if not self.__disponivel:
            self.__disponivel = True
            return True
        return False

    def __str__(self):
        status = "Disponível" if self.disponivel else "Emprestado"
        return f"'{self.titulo}' por {self.autor} | Status: {status}"


# --- 2. CLASSE USUARIO ---
class Usuario:
    def __init__(self, id_usuario: int, nome: str):
        self.__id = id_usuario
        self.__nome = nome.strip()

    @property
    def id(self) -> int:
        return self.__id

    @property
    def nome(self) -> str:
        return self.__nome

    def __str__(self):
        return f"Usuário: {self.nome} (ID: {self.id})"


class Biblioteca:
    """
    Gerencia o acervo de livros e o registro de usuários.
    """

    def __init__(self):
        # Dicionários para busca rápida
        self.__acervo: dict[str, Livro] = {}    # Chave: Título do Livro
        self.__usuarios: dict[int, Usuario] = {}  # Chave: ID do Usuário
        # Rastreamento de empréstimos (Adicionado para um controle mais completo)
        self.__emprestimos: dict[str, int] = {} # Chave: Título do Livro, Valor: ID do Usuário

    # --- MÉTODOS SOLICITADOS ---

    def adicionar_livro(self, livro: Livro):
        """Adiciona um livro ao acervo."""
        titulo = livro.titulo
        if titulo in self.__acervo:
            print(f"ATENÇÃO: Livro '{titulo}' já está no acervo.")
        else:
            self.__acervo[titulo] = livro
            print(f"SUCESSO: Livro '{titulo}' adicionado ao acervo.")

    def adicionar_usuario(self, usuario: Usuario):
        """Registra um novo usuário."""
        user_id = usuario.id
        if user_id in self.__usuarios:
            print(f"ATENÇÃO: Usuário ID {user_id} já está registrado.")
        else:
            self.__usuarios[user_id] = usuario
            print(f"SUCESSO: Usuário '{usuario.nome}' (ID: {user_id}) registrado.")

    # Adicionado para a interface
    def buscar_usuario(self, id_usuario: int) -> Usuario | None:
        return self.__usuarios.get(id_usuario)

    # Adicionado para a interface
    def buscar_livro(self, titulo_livro: str) -> Livro | None:
        return self.__acervo.get(titulo_livro)

    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        """Gerencia o empréstimo."""
        titulo_livro = titulo_livro.strip() # Limpar espaços em branco

        usuario = self.buscar_usuario(id_usuario)
        if not usuario:
            print(f"ERRO: Usuário ID {id_usuario} não encontrado.")
            return

        livro = self.buscar_livro(titulo_livro)
        if not livro:
            print(f"ERRO: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        if livro.emprestar():
            # Rastreia o empréstimo
            self.__emprestimos[titulo_livro] = id_usuario
            print(f"EMPRÉSTIMO SUCESSO: '{livro.titulo}' emprestado para {usuario.nome}.")
        else:
            # Livro já emprestado
            id_quem_emprestou = self.__emprestimos.get(titulo_livro)
            nome_quem_emprestou = self.buscar_usuario(id_quem_emprestou).nome if id_quem_emprestou else "Desconhecido"
            print(f"EMPRÉSTIMO FALHA: '{livro.titulo}' já está emprestado para {nome_quem_emprestou}.")


    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        """Gerencia a devolução."""
        titulo_livro = titulo_livro.strip() # Limpar espaços em branco

        livro = self.buscar_livro(titulo_livro)
        if not livro:
            print(f"ERRO: Livro '{titulo_livro}' não encontrado para devolução.")
            return

        if livro.devolver():
            # Remove o rastreamento do empréstimo
            if titulo_livro in self.__emprestimos:
                # Opcional: Verificar se o usuário que devolve é o mesmo que pegou emprestado
                id_quem_pegou = self.__emprestimos.pop(titulo_livro)
                # Adicionar uma checagem seria útil para um sistema robusto:
                # if id_quem_pegou != id_usuario:
                #     print("ATENÇÃO: Livro devolvido por um usuário diferente do original.")

            print(f"DEVOLUÇÃO SUCESSO: '{livro.titulo}' devolvido à biblioteca.")
        else:
            print(f"DEVOLUÇÃO FALHA: '{livro.titulo}' já estava disponível. Verifique o título.")

    # --- Método de Visualização Simples ---
    def listar_acervo(self):
        """Lista o estado atual de todos os livros no acervo."""
        print("\n--- LISTA DO ACERVO ---")
        if not self.__acervo:
            print("Acervo vazio.")
            return

        # Ordena por título para melhor visualização
        livros_ordenados = sorted(self.__acervo.values(), key=lambda livro: livro.titulo)

        for livro in livros_ordenados:
            emprestado_para = ""
            if not livro.disponivel:
                id_usuario = self.__emprestimos.get(livro.titulo)
                usuario = self.buscar_usuario(id_usuario)
                emprestado_para = f" (para {usuario.nome} - ID: {id_usuario})" if usuario else " (Para Usuário Desconhecido)"

            print(f"- {livro}{emprestado_para}")

    def listar_usuarios(self):
        """Lista todos os usuários registrados."""
        print("\n--- LISTA DE USUÁRIOS ---")
        if not self.__usuarios:
            print("Nenhum usuário registrado.")
            return
        for usuario in self.__usuarios.values():
            print(f"- {usuario}")


# --- 5. INTERFACE DE CONSOLE ---
def exibir_menu():
    """Exibe as opções do menu principal."""
    print("\n" + "="*40)
    print("      SISTEMA DE GESTÃO DA BIBLIOTECA")
    print("="*40)
    print("1. Adicionar Novo Livro")
    print("2. Registrar Novo Usuário")
    print("3. Emprestar Livro")
    print("4. Devolver Livro")
    print("5. Listar Acervo (Livros)")
    print("6. Listar Usuários")
    print("7. Sair")
    print("-" * 40)

def main_interface():
    """Função principal da interface do console."""
    biblioteca = Biblioteca()

    # --- PRÉ-CARREGAR DADOS (Opcional, para facilitar testes) ---
    biblioteca.adicionar_livro(Livro("Harry Potter e a Pedra Filosofal", "J. K. Rowling"))
    biblioteca.adicionar_livro(Livro("Dom Quixote", "Miguel de Cervantes"))
    biblioteca.adicionar_usuario(Usuario(10, "Maria"))
    biblioteca.adicionar_usuario(Usuario(20, "João"))
    print("\n[Dados iniciais carregados para demonstração]")
    # -----------------------------------------------------------

    while True:
        exibir_menu()
        escolha = input("Escolha uma opção (1-7): ")

        if escolha == '1':
            print("\n--- ADICIONAR LIVRO ---")
            titulo = input("Título do Livro: ").strip()
            autor = input("Autor do Livro: ").strip()
            if titulo and autor:
                biblioteca.adicionar_livro(Livro(titulo, autor))
            else:
                print("ERRO: Título e Autor não podem ser vazios.")

        elif escolha == '2':
            print("\n--- REGISTRAR USUÁRIO ---")
            try:
                id_usuario = int(input("ID do Usuário (apenas números): ").strip())
                nome = input("Nome do Usuário: ").strip()
                if nome:
                    biblioteca.adicionar_usuario(Usuario(id_usuario, nome))
                else:
                    print("ERRO: O nome do usuário não pode ser vazio.")
            except ValueError:
                print("ERRO: ID do usuário deve ser um número inteiro.")

        elif escolha == '3':
            print("\n--- EMPRESTAR LIVRO ---")
            try:
                id_usuario = int(input("ID do Usuário que está emprestando: ").strip())
                titulo_livro = input("Título do Livro a ser emprestado: ").strip()
                biblioteca.emprestar_livro(id_usuario, titulo_livro)
            except ValueError:
                print("ERRO: ID do usuário deve ser um número inteiro.")

        elif escolha == '4':
            print("\n--- DEVOLVER LIVRO ---")
            try:
                id_usuario = int(input("ID do Usuário que está devolvendo: ").strip()) # Necessário para checagens futuras
                titulo_livro = input("Título do Livro a ser devolvido: ").strip()
                biblioteca.devolver_livro(id_usuario, titulo_livro)
            except ValueError:
                print("ERRO: ID do usuário deve ser um número inteiro.")

        elif escolha == '5':
            biblioteca.listar_acervo()

        elif escolha == '6':
            biblioteca.listar_usuarios()

        elif escolha == '7':
            print("\nObrigado por usar o Sistema da Biblioteca. Saindo...")
            break

        else:
            print("Opção inválida. Por favor, escolha um número de 1 a 7.")

        input("\nPressione ENTER para continuar...") # Pausa para o usuário ler a saída


if __name__ == '__main__':
    main_interface()

SUCESSO: Livro 'Harry Potter e a Pedra Filosofal' adicionado ao acervo.
SUCESSO: Livro 'Dom Quixote' adicionado ao acervo.
SUCESSO: Usuário 'Maria' (ID: 10) registrado.
SUCESSO: Usuário 'João' (ID: 20) registrado.

[Dados iniciais carregados para demonstração]

      SISTEMA DE GESTÃO DA BIBLIOTECA
1. Adicionar Novo Livro
2. Registrar Novo Usuário
3. Emprestar Livro
4. Devolver Livro
5. Listar Acervo (Livros)
6. Listar Usuários
7. Sair
----------------------------------------
Escolha uma opção (1-7): 2

--- REGISTRAR USUÁRIO ---
ID do Usuário (apenas números): 6
Nome do Usuário: Pietro Fleith
SUCESSO: Usuário 'Pietro Fleith' (ID: 6) registrado.

Pressione ENTER para continuar...

      SISTEMA DE GESTÃO DA BIBLIOTECA
1. Adicionar Novo Livro
2. Registrar Novo Usuário
3. Emprestar Livro
4. Devolver Livro
5. Listar Acervo (Livros)
6. Listar Usuários
7. Sair
----------------------------------------
